In [ ]:
import numpy as np                     # for random number generation
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
"""
Generated a set of fake data of value of a fund index with 365 daily quote each year, 
from date 2015.12.31 to 2036.12.31. The value of fund index as of 2015.12.31 is 100.00.
The values van be generated randomly but:
(Fund A,1) During 2016.01.01 to 2025.12.31 the average annual return is 15%
    and the volatility, standard deviation of annual return is 18%
(Fund A,2) During 2026.01.01 to 2036.12.31 the average annual return is 10%
    and the volatility, standard deviation of annual return is 11%
(Fund B,1) During 2016.01.01 to 2025.12.31 the average annual return is 10%
    and the volatility, standard deviation of annual return is 12%
(Fund B,2) During 2026.01.01 to 2036.12.31 the average annual return is 8%
    and the volatility, standard deviation of annual return is 10%
(Fund C,1) During 2016.01.01 to 2025.12.31 the average annual return is 18%
    and the volatility, standard deviation of annual return is 22%
(Fund C,2) During 2026.01.01 to 2036.12.31 the average annual return is 15%
    and the volatility, standard deviation of annual return is 18%
(Fund D, TimeDeposit) During 2016.01.01 to 2036.12.31 the average annual return is 3%
    and the volatility, standard deviation of annual return is 0.7%, daily return must be non-negative

"""
# Generate date range from 2015-12-31 to 2036-12-31 (includes leap days automatically)

# ───────────────────────────────────────────────────────────────
# CONFIGURATION - Four Funds now (A, B, C, D)
# ───────────────────────────────────────────────────────────────

START_DATE = datetime(2015, 12, 31)
END_DATE   = datetime(2036, 12, 31)
BASE_VALUE = 100.00

# Regime change date (used by A,B,C)
CHANGE_DATE = datetime(2026, 1, 1)

# Fund parameters
fund_configs = {
    'A': {
        '2016-2025': {'return': 0.15, 'vol': 0.18},
        '2026-2036': {'return': 0.10, 'vol': 0.11}
    },
    'B': {
        '2016-2025': {'return': 0.10, 'vol': 0.12},
        '2026-2036': {'return': 0.08, 'vol': 0.10}
    },
    'C': {
        '2016-2025': {'return': 0.18, 'vol': 0.22},
        '2026-2036': {'return': 0.15, 'vol': 0.18}
    },
    'D': {
        'whole_period': {'return': 0.03, 'vol': 0.007}  # 0.7% annual vol
    }
}

# ───────────────────────────────────────────────────────────────
# Generate date range
# ───────────────────────────────────────────────────────────────
dates = pd.date_range(START_DATE, END_DATE, freq='D')

# Store price series
prices = {}

np.random.seed(42)  # reproducible - remove/change for different paths

for fund_name, config in fund_configs.items():
    price_series = pd.Series(index=dates, dtype=float)
    price_series.iloc[0] = BASE_VALUE

    if fund_name == 'D':
        # Special case: Fund D - low vol, no negative daily returns
        r = config['whole_period']['return']
        v = config['whole_period']['vol']
        mu_annual = np.log(1 + r) - 0.5 * v**2
        mu_daily = mu_annual / 365
        sigma_daily = v / np.sqrt(365)

        for i in range(1, len(dates)):
            # Generate normal random return, but floor at 0
            daily_log_ret = np.random.normal(mu_daily, sigma_daily)
            daily_log_ret = max(daily_log_ret, 0)  # No negative daily returns
            
            price_series.iloc[i] = price_series.iloc[i-1] * np.exp(daily_log_ret)

    else:
        # Normal funds A,B,C with regime change
        regimes = {}
        for period, params in config.items():
            r = params['return']
            v = params['vol']
            mu_annual = np.log(1 + r) - 0.5 * v**2
            regimes[period] = {
                'mu_daily': mu_annual / 365,
                'sigma_daily': v / np.sqrt(365)
            }

        for i in range(1, len(dates)):
            dt = dates[i]
            if dt < CHANGE_DATE:
                mu_d = regimes['2016-2025']['mu_daily']
                sigma_d = regimes['2016-2025']['sigma_daily']
            else:
                mu_d = regimes['2026-2036']['mu_daily']
                sigma_d = regimes['2026-2036']['sigma_daily']
            
            log_ret = np.random.normal(mu_d, sigma_d)
            price_series.iloc[i] = price_series.iloc[i-1] * np.exp(log_ret)

    prices[fund_name] = price_series


# ───────────────────────────────────────────────────────────────
# Query Functions
# ───────────────────────────────────────────────────────────────

def get_fund_value(fund: str, date_str: str) -> None:
    """Print fund name, date and value"""
    fund = fund.upper()
    if fund not in prices:
        print(f"Error: Fund '{fund}' not found. Use 'A','B','C','D'")
        return
    
    try:
        dt = pd.to_datetime(date_str)
        value = prices[fund].loc[dt]
        print(f"Fund {fund} | Date: {dt.strftime('%Y-%m-%d')} | Value: {value:,.2f}")
    except KeyError:
        print(f"Error: Date '{date_str}' not in range (2015-12-31 ~ 2036-12-31)")


def get_fund_values_range(fund: str, start_str: str, end_str: str) -> None:
    """Print values for a date range"""
    fund = fund.upper()
    if fund not in prices:
        print(f"Error: Fund '{fund}' not found. Use 'A','B','C','D'")
        return
    
    try:
        start = pd.to_datetime(start_str)
        end   = pd.to_datetime(end_str)
        
        series = prices[fund].loc[start:end]
        if series.empty:
            print("No data in the selected range.")
            return
            
        print(f"\nFund {fund} values from {start_str} to {end_str}:\n")
        for date, value in series.items():
            print(f"{date.strftime('%Y-%m-%d')}: {value:,.2f}")
            
    except KeyError:
        print("Error: One or both dates are out of range.")


# ───────────────────────────────────────────────────────────────
#                          EXAMPLES
# ───────────────────────────────────────────────────────────────

if __name__ == "__main__":
    print("=== Single Date Examples ===")
    get_fund_value('A', '2025-12-31')
    get_fund_value('B', '2025-12-31')
    get_fund_value('C', '2025-12-31')
    get_fund_value('D', '2025-12-31')   # ← Fund D added
    get_fund_value('D', '2036-12-31')
    
    print("\n=== Range Example - Fund D (very stable) ===")
    get_fund_values_range('D', '2030-01-01', '2030-01-10')


=== Single Date Examples ===
Fund A | Date: 2025-12-31 | Value: 829.33
Fund B | Date: 2025-12-31 | Value: 230.04
Fund C | Date: 2025-12-31 | Value: 858.15
Fund D | Date: 2025-12-31 | Value: 198.45
Fund D | Date: 2036-12-31 | Value: 422.56

=== Range Example - Fund D (very stable) ===

Fund D values from 2030-01-01 to 2030-01-10:

2030-01-01: 262.24
2030-01-02: 262.24
2030-01-03: 262.24
2030-01-04: 262.24
2030-01-05: 262.24
2030-01-06: 262.38
2030-01-07: 262.40
2030-01-08: 262.40
2030-01-09: 262.40
2030-01-10: 262.40
